In [13]:
### a web scraping project by jane n tom ###

import urllib.parse
import time
import requests
import json

base = 'https://www.alltrails.com/'
# api/alltrails/locations/states/50/trails
# ?page=2per_page=100&algolia_formatted=true&country_id=313
api = '3p0t5s6b5g4g0e8k3c1j3w7y5c3m4t8i'
wait = 0.1

In [14]:
# create a function to automate page loading
def getStateTrails(stateid, wait = wait, timeout_wait_base = 10):
    
    path = f'api/alltrails/locations/states/{stateid}/trails'
    url = urllib.parse.urljoin(base, path)
    headers = {
        'x-at-key': api
            }
    page = 1
    timeout_retry_count = 0
    while True:
        print(f'page {page}')
        params = {
            'page': page,
            'per_page': 100,
            'algolia_formatted': True,
            'country_id': 313
        }
        try:
            respond = requests.get(
                    url, 
                    params = params, 
                    headers=headers, 
                    timeout=10 )
        except requests.exceptions.ReadTimeout:
            timeout_wait = timeout_wait_base * (2 ** timeout_retry_count)
            print(f'API call timed out, waiting {timeout_wait}')
            time.sleep(timeout_wait)
            timeout_retry_count += 1
            continue
                
        timeout_retry_count= 0 #if backoff ok, reset to base wait
        
        if not respond.ok:
            raise Exception(
                f'Bad response from API, state {stateid}, page:{page}'
            )
            
        trails = respond.json()['trails']
        
        if not trails:
            break
        yield from trails
        
        time.sleep(wait)
        page += 1
        
#1600ish expected trails
#trails = getStateTrails(39) #rhode island

In [20]:
# testing the function on hawaii
# sidenote: I need to go hiking in hawaii someday, these trails are unreal
def getHawaii(hawaii, wait = wait, timeout_wait_base = 10):
    
    path = f'api/alltrails/locations/countries/{hawaii}/trails'
    url = urllib.parse.urljoin(base, path)
    headers = {
        'x-at-key': api
            }
    page = 1
    print(1)
    timeout_retry_count = 0
    while True:
        print(f'page {page}')
        params = {
            'page': page,
            'per_page': 100,
            'algolia_formatted': True,
        }
        try:
            respond = requests.get(
                    url, 
                    params = params, 
                    headers=headers, 
                    timeout=10 )
            print(respond)
        except requests.exceptions.ReadTimeout:
            timeout_wait = timeout_wait_base * (2 ** timeout_retry_count)
            print(f'API call timed out, waiting {timeout_wait}')
            time.sleep(timeout_wait)
            timeout_retry_count += 1
            continue
                
        timeout_retry_count= 0 #if backoff ok, reset to base wait
        
        if not respond.ok:
            raise Exception(
                f'Bad response from API, state {stateid}, page:{page}'
            )
            
        trails = respond.json()['trails']
        
        if not trails:
            break
        yield from trails
        
        time.sleep(wait)
        page += 1

In [21]:
#trails = getStateTrails(39) #penn
trails = getHawaii(366)

In [17]:
# put it into action as it loops through each state, let it run for awhile
trails = []
for i in range (1, 50 +1):
   trails = list(getStateTrails(i))   

page 1


Exception: Bad response from API, state 1, page:1

In [11]:
next(trails)

TypeError: 'list' object is not an iterator

In [42]:
# separate the files by state, when state is completed save the file 
allTrails = {}

for i in range(51,52): #alphabetical states
    trails = list(getStateTrails(i, 0))
    state = trails[0]['state_name']
    with open(f'./State Trails Data/{state}.json', 'w+') as out_file:
        out_file.write(json.dumps(trails))
        print(f'State {i} ({state}) done, {len(trails)} trails')
    
    allTrails[i] = trails
    #input('next >')

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
State 51 (Wyoming) done, 711 trails


In [38]:
trail = trails[0]
trail

{'objectID': 'trail-10026910',
 'ID': 10026910,
 'slug': 'us/california/vernal-and-nevada-falls-via-the-mist-trail',
 'name': 'Vernal and Nevada Falls via the Mist Trail',
 'popularity': 68.1488,
 'type': 'trail',
 '_geoloc': {'lat': 37.73495, 'lng': -119.56619},
 'length': 10299.776,
 'elevation_gain': 667.8168000000001,
 'difficulty_rating': '3',
 'route_type': 'L',
 'visitor_usage': '3',
 'avg_rating': 5.0,
 'area_id': 10106838,
 'area_name': 'Yosemite National Park',
 'country_id': 313,
 'country_name': 'United States',
 'city_id': 1036,
 'city_name': 'Yosemite Valley',
 'state_id': 5,
 'state_name': 'California',
 'verified_map_id': 22213085,
 'features': ['dogs-no',
  'forest',
  'partially-paved',
  'river',
  'views',
  'waterfall',
  'wild-flowers',
  'wildlife'],
 'activities': ['birding', 'hiking', 'nature-trips', 'walking'],
 'feature_names': ['Dogs No',
  'Forest',
  'Partially Paved',
  'River',
  'Views',
  'Waterfall',
  'Wild Flowers',
  'Wildlife'],
 'activity_names':

In [47]:
# test it on hawaii again
trails = list(getHawaii(366, 0))
state = trails[0]['state_name']
with open(f'./State Trails Data/Hawaii.json', 'w+') as out_file:
    out_file.write(json.dumps(trails))
    print(f'Hawaii ({state}) done, {len(trails)} trails')

page 1
page 2
page 3
page 4
page 5
page 6
page 7
State 51 (Oahu) done, 528 trails


In [50]:
### testing another web scraping method using beautifulsoup ###

In [53]:
state = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

In [54]:
# scraping the site 
def get_all_hikes(browser):
    for st in state:
        browser.get(f'https://www.alltrails.com/us/{state}')
        while True:
            try:
                load_hikes = WebDriverWait(browser, 20).until(EC.visibility_of_element_located((By.XPATH,"//div[@id='load_more'] [@class='feed-item load-more trail-load'][//a]")))
                load_more_hikes.click()
                time.sleep(7)
            except:
                break
        soup = BeautifulSoup(browser.page_source)
        return soup

In [52]:
# scraping the details
def parse_meta_data(hike_soup):
    header = hike_soup.find('div', id='title-and-menu-box')
    hike_name = header.findChild('h1').text
    difficulty = header.findChild('span').text
    stars = header.findChild('meta')['content']
    num_reviews = header.find('span', itemprop='reviewCount').text
    area = hike_soup.select('div.trail-rank')
    try:
        hike_region = area[0].findChild('span', itemprop='name').text
    except:
        hike_region = area[0].findChild('a').text
    # directions = header.select('li.bar-icon.trail-directions')
    try:
        distance = hike_soup.select('span.distance-icon')[0].text
    except:
        distance = None
    try:
        elevation_gain = hike_soup.select('span.elevation-icon')[0].text
    except:
        elevation_gain = None
    try:
        route_type = hike_soup.select('span.route-icon')[0].text
    except:
        route_type = None
    tags = hike_soup.select('section.tag-cloud')[0].findChildren('h3')
    hike_attributes = []
    for tag in tags:
        hike_attributes.append(tag.text)
    user_ratings = []
    users = hike_soup.select('div.feed-user-content.rounded')
    for user in users:
        if user.find('span', itemprop='author') != None:
            user_name = user.find('span', itemprop='author').text
            user_name = user_name.replace('.', '')
            try:
                rating = user.find('span', itemprop="reviewRating").findChildren('meta')[0]['content']
                user_ratings.append({user_name: rating})
            except:
                pass
    row_data = {}
    row_data['hike_name'] = hike_name
    row_data['hike_difficulty'] = difficulty
    row_data['stars'] = stars
    row_data['num_reviews'] = num_reviews
    row_data['hike_region'] = hike_region
    row_data['total_distance'] = distance
    row_data['elevation_gain'] = elevation_gain
    row_data['route_type'] = route_type
    row_data['hike_attributes'] = hike_attributes
    row_data['ratings'] = user_ratings
    return row_data

In [9]:
browser = webdriver.Chrome()
soup = get_all_hikes(browser)
create_db(soup, browser)